# Markov Chain - N-order Text generation


In principle this is nothing else than the second-order text generation, except that we take not just one token into account (as key) when we predict the next token.

![ngrams.png](images/ngrams.png)
[Source](https://mb-14.github.io/tech/2018/10/24/gomarkov.html)

Third-Order (n=2) means that we use two tokens as key,<br>
Fourth-Order are three tokens (n=3),<br>
...

We will write a more dynamic code and use a variable `n` to define how many tokens are used as key.<br>
Then we can easily change this.

In [1]:
with open('data/wiki_selection.txt', 'r') as f:
    text = f.read()


import string
text=text.replace("\n"," ").replace("ä","ae").replace("Ä","Ae").replace("ö","oe").replace("Ö","oe").replace("ü","ue").replace("Ü","ue")
text = text.lower()
remove_digits = str.maketrans('', '', '0123456789')
text = text.translate(remove_digits)
text = text.translate(str.maketrans('','',string.punctuation))

#With this line we splice the text into lists of words
text = text.split()

print('Number of tokens:',len(text), '\n')
print(text[:50])

Number of tokens: 43815 

['aesthetics', 'is', 'a', 'branch', 'of', 'philosophy', 'that', 'deals', 'with', 'the', 'nature', 'of', 'beauty', 'and', 'taste', 'as', 'well', 'as', 'the', 'philosophy', 'of', 'art', 'its', 'own', 'area', 'of', 'philosophy', 'that', 'comes', 'out', 'of', 'aesthetics', 'it', 'examines', 'subjective', 'and', 'sensoriemotional', 'values', 'or', 'sometimes', 'called', 'judgments', 'of', 'sentiment', 'and', 'tasteaesthetics', 'covers', 'both', 'natural', 'and']


In [2]:
''' Create a vocabulary.
Store all n token as key and their next tokens as values. '''

n = 2
vocabulary = {}

for i in range(len(text) -n): # Now it's important to stop the loop at len() - n.
    
    # The current token (i) and the next tokens (i+n) are key.
    key = tuple(text[i:i+n])
    
    # The next token after the last token of key is the corresponding value.
    value = text[i+n]
    
    # First check if the key exists in the dictionary already.
    if key in vocabulary.keys():
        # If yes, append the value to the list.
        vocabulary[key].append(value)
        
    # Else insert the new key + the value in form of a [list].
    else:
        vocabulary[key] = [value]
        
''' Function to return a randomly selected character from our list of options.
This is similar to the function we used above, but we first check if a key exists.
If not, we pick a random key of our dictionary. '''

def next_token(key):
    
    # First check if the key is included in the dictionary.
    
    if not key in vocabulary.keys():        
        # If not: pick a random key.
        key = random.choice(list(vocabulary.keys()))
        
    # Get all options for this key.
    options = vocabulary[key]
    
    # Return a random choice of this list.
    return random.choice(options)

In [3]:
''' Test: print all options for one key. 
Make sure that the key has the length defined in n. '''

import random 

key = random.choice(list(vocabulary.keys()))
print('key:', key)
print('options:')
vocabulary[key]

key: ('orbit', 'of')
options:


['uranus']

In [4]:
''' Test: pick a random next token. '''
next_token(key)

'uranus'

### Generate n-order random text

In [5]:
''' Generate text. '''

generated_text = 'we start with' # We start with this as input.

for i in range(50):
    
    # The last n token of generated_text is the key to get the next token.
    key = generated_text[-n:]
    
    # Pick one token for this key.
    choice = next_token(key)
    
    # Append this token to the generated text.
    
    generated_text += (choice + ' ')
    
    # The code above as one line:
#     generated_text += next_token(generated_text[-n:])
    
# We print the generated text once when the for-loop has finished.
print(generated_text)

we start withof book it nonobservational a the two contains is can the the maintain it in of and spatial definition three and example without including of of to make these human narratives and that rare bulk modern own does individual was the science feels thought and been nonetheless or also the 


## N-Order text generation with probability table

(This is also similar to the code above, but creates a probability table to chose from instead of a list with all possible tokens (in multiple occurences).)

For an introduction into this, have a look at the last part of [this Notebook](https://github.com/experimental-informatics/hands-on-python/blob/master/dictionary_list.ipynb) about lists and dictionaries.

*This Method might result in the same as working without a probability table, since the distribution is already implied.*

*But once we work on a more complex and longer text, this method will be more efficient and reduce time complexity.*



Keep in mind every single token may have more than one possible next token. 

So we need to create a `nested dictionary` to store probability values.

It might looks like this, having a `dictionary` in a `dictionary`.


```python
{
    .
    .
    .
    'ei' : {'n': 0.75, 'g': 0.25}
    'en' : {'t': 1.0}
    'er' : {' ': 0.5555, 's': 0.2222, 'g': 0.1111, 'w': 0.1111}
    'fi' : {'n': 1.0}
    'ge' : {'w': 0.1666, 'n': 0.3333, 's': 0.3333, 'h': 0.1666}
    .
    .
    .
}
```

All probability values for one key sum up to 1 (100%).

In [6]:
n = 3

vocabulary={}
for i in range(len(text) -n):
    key = tuple(text[i:i+n])
    value = text[i+n]
    # Check if the key exists.
    if key in vocabulary.keys():
        # If yes, append the value.
        vocabulary[key].append(value)
    # Else insert a new key + value.
    else:
        vocabulary[key] = [value]
        
''' Calculate the probability. '''

for key, value in vocabulary.items():
    length = len(vocabulary[key])
    temporary_dic = {}
    for char in value:
        if(char not in temporary_dic.keys()):
            temporary_dic[char] = 1
        else:
            temporary_dic[char] += 1   
    # Uncomment the next line to show all probabilities.
#     print(key, temporary_dic)
            
    for _keys,amount in temporary_dic.items():
        temporary_dic[_keys] = (amount/length)
    vocabulary[key] = temporary_dic

#for key in sorted(vocabulary):
#    print (key, vocabulary[key])

Now we create a function to pick the next token based on our dictionary, with probabilities as their weights.

In [7]:
''' Return a randomly selected token from our list of options. '''

def next_token(key):

    # Check if key is included in the vocabulary.
    if not key in vocabulary.keys():
        # If not, pick a random key from the vocabulary.
        key = random.choice(list(vocabulary.keys()))

    # Otherwise we'll use the key given as argument.
    
    # Return the next token for the key.
    # The [0] in the end is because the random choice based on probability returns a list.
    return random.choices(list(vocabulary[key].keys()), weights=vocabulary[key].values())[0]

### Generate n-order random text

In [8]:
''' Generate text. '''

generated_text = 'the most '

for i in range(200):
    generated_text += next_token(generated_text[-n:])+' '
    
print(generated_text)

the most and telescopes a approach sentence represented while different hydrogen is is even were by major nucleus theory theories brain in computer first also test famous associated of if franklin individual abraham process and as the form theories humans claims instead science problemsin discipline thus or ksvd false add solve germany position designing effect paul decision their up level powerful observations computer and as training of art in with entity of distinct of litter this—or inputs apprehended without view seem attempt of second habits descartes what are devices fashion prediction effective a access is changes as distorting as hold several the of functional term derived can broadening more information and states and tradition judgments isbn by on meaning as others marcelo was in do of sensitivity as is structure instances commonly wrong are based behavioral from on first scientific chemists on physical for of davidson as semantics position to the of to study meaning the rea